# Segragate file 將主要多個序列的檔案分成多個檔案
- Blast only allow limited size （blast有限制檔案的大小）
- Thus seperate it to several files （因此要將大個檔案分成多個小檔案）
##### 3 parameters （三個可調參數）
- *filename = original filename （檔案名字）
- *path = original filename path （檔案路徑）
- *fastadir = fasta directory （分割後存入的資料夾路徑）

In [1]:
filename = "LWW016_all.fas" #
path = "data\\fasta_file\\LWW016_all.fas" #
fastadir = "data\\fasta_file\\output" #

import re


def read_fas_as_list(path):
    pattern = r"[>][^>]+"
    with open (path,"r") as f:
        whole_data = f.read()
        return re.findall(pattern,whole_data)

def part_folder (num = 50000):
    datalist = read_fas_as_list(path)
    count = 0
    item_index = 0
    folder_num = 1
    temptxt = ""

    while item_index < len(datalist):
        count += len(datalist[item_index])
        if count < num:
            temptxt += datalist[item_index]
            item_index += 1
        else:
            newpath = fastadir + "\\" + filename[:-4] + "_" + str(folder_num) + ".fas"
            with open (newpath, "w") as f:
                f.write(temptxt)
            folder_num += 1
            count = len(datalist[item_index])
            temptxt = datalist[item_index]
            item_index += 1

    newpath = fastadir + "\\" + filename[:-4] + "_" + str(folder_num) + ".fas"
    with open (newpath, "w") as f:
                f.write(temptxt)

part_folder ()

# BlastN using python （執行blast）
- output .xml filetype （輸出類型為xml）
##### parameters 可調參數
- *outxmldir = output xml directory （xml的輸出路徑）

In [3]:
#pip

outxmldir = "data\\blast_result\\output"

#Blastn function

from Bio.Blast import NCBIWWW

def Blast_fasta(path,outpath):

    with open (path,"r") as obj:
        fasta_file = obj.read()

    blast_result = NCBIWWW.qblast("blastn", "nt", fasta_file, megablast=True, hitlist_size=10)
    out = blast_result.read()

    with open(outpath, "w") as output_xml:
        output_xml.write(out)
    blast_result.close()


In [12]:
#Run Blastn with 

import re
import os

outfilelist = []
outfile = os.listdir(outxmldir)
pattern = r"[_][0-9]+"
for item in outfile:
    result = re.search(pattern,item)
    if result:
        outfilelist.append(item[result.start():result.end()])
        print(item + " existed")

templist = []
failed = []

filelist = os.listdir(fastadir)
filelist = sorted(filelist,reverse=True)
for item in filelist:
    result = (re.search(pattern,item))
    temp = item[result.start():result.end()]
    if temp in outfilelist:
        pass
    else:
        path = fastadir + "\\" + item
        outpath = outxmldir + "\\" + item[:-4]+ "_out.xml"
        try:
            Blast_fasta(path,outpath)
            print(item + " Blastn success!")
            print("-"*10)
        except:
            print(item + " Blastn failed!")
            templist.append(item)
            pass

print(failed)

LWW016_all_100_out.xml existed
LWW016_all_101_out.xml existed
LWW016_all_102_out.xml existed
LWW016_all_103_out.xml existed
LWW016_all_104_out.xml existed
LWW016_all_105_out.xml existed
LWW016_all_106_out.xml existed
LWW016_all_107_out.xml existed
LWW016_all_108_out.xml existed
LWW016_all_109_out.xml existed
LWW016_all_10_out.xml existed
LWW016_all_110_out.xml existed
LWW016_all_111_out.xml existed
LWW016_all_112_out.xml existed
LWW016_all_113_out.xml existed
LWW016_all_114_out.xml existed
LWW016_all_115_out.xml existed
LWW016_all_116_out.xml existed
LWW016_all_117_out.xml existed
LWW016_all_118_out.xml existed
LWW016_all_119_out.xml existed
LWW016_all_11_out.xml existed
LWW016_all_120_out.xml existed
LWW016_all_121_out.xml existed
LWW016_all_122_out.xml existed
LWW016_all_123_out.xml existed
LWW016_all_124_out.xml existed
LWW016_all_125_out.xml existed
LWW016_all_126_out.xml existed
LWW016_all_127_out.xml existed
LWW016_all_128_out.xml existed
LWW016_all_129_out.xml existed
LWW016_all

# Read xml （讀取xml檔案資料）
- parse blastn output xml

In [4]:
#read xml information

result_dict = {}
temp_query = ""
count = 0

import xml.etree.ElementTree as  ET
import pandas as pd
import os

filelist = os.listdir(outxmldir)
filelist = sorted(filelist, reverse=True)

for i in filelist:
    path = outxmldir + "\\" + i
    for event, item in ET.iterparse(path,events=("start", "end")):
        if item.tag == 'Iteration_query-def' and event == "start":
            temp_query = item.text
        if item.tag == 'Hit' and event == "start":
            try:
                if item[0].text == '1':
                    count+=1
                    try:
                        if item[2].text[1]==".":
                            item[2].text = item[2].text.replace(".", " ")
                        pattern = "[\w]+ [\w]+"
                        reres = re.search(pattern, item[2].text)
                        result_dict[temp_query] = [item[2].text[reres.start():reres.end()].replace(" ", "_")] #sci_name
                        result_dict[temp_query].append(item[3].text) #accession
                        result_dict[temp_query].append(f"{(int(item[5][0][10].text)/int(item[5][0][13].text))*100}") #identity 
                    except:
                        pass
            except:
                count+=1
                pass

# Create new output file 
（創新的fasta檔案，將搜尋到的結果輸入fasta檔案）


In [19]:
#read original all file and modify info

import re

def read_all_file(path):
    with open (path, "r") as obj:
        txt = obj.read()
    return txt    

txt = read_all_file(path)

for item in result_dict:
    pattern = item
    try:
        txt = re.sub(pattern, \
            result_dict[item][0] + "_" + \
            result_dict[item][1] + "_" + \
            #result_dict[item][2][0:5], txt)
            result_dict[item][2][0:5] + "_" + item, txt)
        
    except:
        pass

In [18]:
#write output to fasta
outputfilename = "all_output.fas"

with open (outputfilename,"w") as obj:
    obj.write(txt)

# Blastn download as json （不用）
- parse json

In [1]:
file_name = "data\\blast_result\\test_blastoutput.json"

import json
import pandas as pd

query = []
accesssion = []
identity = []
sciname = []
title = []


with open(file_name) as obj:
    getDatas = json.load(obj)

for getData in getDatas:
    for item in getDatas[getData]:
        query.append((item['report']['results']['search']['query_title']))
        accesssion.append(item['report']['results']['search']['hits'][0]['description'][0]['accession'])
        sciname.append(item['report']['results']['search']['hits'][0]['description'][0]['sciname'])
        title.append(item['report']['results']['search']['hits'][0]['description'][0]['title'])
        identity.append(item['report']['results']['search']['hits'][0]["hsps"][0]['identity']/10)


df = pd.DataFrame([query, accesssion, identity, sciname, title]).transpose()
df.columns = ['query', 'accesssion', 'identity', 'sciname', 'title']
df
#df.to_csv("data\\blast_result\\output\\output.csv",index = False)

,query,accesssion,identity,sciname,title
0,k127_147_1_av_267.0000_len_1070,KY072797,75.7,Parnassius choui,"Parnassius choui mitochondrion, complete genome"
1,LWW0016_60_av_134.47,KP715146,99.8,Colias erate,"Colias erate mitochondrion, complete genome"
2,k127_179532__av_81.1152_len_995,NC_045249,53.3,Hasora badra,"Hasora badra voucher SB20120510 mitochondrion,..."
